<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/160_LangGraph_Intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## 🧠 Overview: LangGraph for Agent Recipes

LangGraph is a **stateful, multi-step agent orchestration framework** built on top of LangChain. It is especially strong for:

* Agent workflows with conditional logic
* Stateful memory (via state graphs)
* Reversible + resumable steps
* Tool integration with planning and control
* Forking and branching execution

---

## 🧩 Mapping Your Agent Recipe to LangGraph

| Your Recipe Step                  | LangGraph Equivalent                                      |
| --------------------------------- | --------------------------------------------------------- |
| 1. Define Agent’s Purpose         | Entry point in the graph (`entry_point="start"`)          |
| 2. Identify Required Capabilities | Nodes: `create_plan`, `track_progress`, domain tools      |
| 3. List Tool Dependencies         | Shared state and `context` in LangGraph's `State` class   |
| 4. Implement Tools                | LangGraph `Tool` nodes, or just Python callables          |
| 5. Build Tool Registry            | Not needed — LangGraph nodes + routing handles this       |
| 6. Assemble ActionContext         | Define your `State` object and inject deps into nodes     |
| 7. Create Environment             | LangGraph handles this via `Graph` and `StateGraph`       |
| 8. Compose the Agent              | LangGraph `StateGraph` with `add_node`, `add_conditional` |
| 9. Wiring                         | Graph wiring with `add_edge`, `set_entry_point`           |
| 10. Test & Iterate                | `.compile()` + `.invoke()` or `.stream()`                 |

---



## 🏗️ LangGraph Skeleton Aligned with Your Recipe

Here’s a minimal implementation following your agent pattern but using LangGraph:

### 1. Define Your Agent’s State

```python
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    goal: str
    plan: Optional[List[str]]
    progress: List[Dict[str, Any]]
    step_index: int
```

---

### 2. Define Your Tools as Nodes

```python
def create_plan_node(state: AgentState) -> AgentState:
    plan = [
        f"Understand the goal: {state['goal']}",
        "Choose one tiny next action",
        "Do the action and review",
    ]
    return {**state, "plan": plan}

def track_progress_node(state: AgentState) -> AgentState:
    progress_entry = {
        "step": state["step_index"],
        "status": "in_progress",
        "note": "Started.",
    }
    updated = state.get("progress", []) + [progress_entry]
    return {**state, "progress": updated}

def finish_node(state: AgentState) -> str:
    return "end"
```

---

### 3. Build the Graph (Wiring & Capabilities)

```python
builder = StateGraph(AgentState)

builder.add_node("create_plan", create_plan_node)
builder.add_node("track_progress", track_progress_node)
builder.add_node("finish", finish_node)

builder.set_entry_point("create_plan")
builder.add_edge("create_plan", "track_progress")
builder.add_edge("track_progress", "finish")
builder.add_edge("finish", END)

graph = builder.compile()
```

---

### 4. Run the Agent

```python
result = graph.invoke({
    "goal": "Practice the agent loop by making a tiny plan.",
    "progress": [],
    "step_index": 0,
})
print(result)
```

---

## ⚙️ Optional: PlanFirst + ProgressTracking Capabilities

These can be turned into:

* **Subgraphs** (for injecting behavior like lifecycle hooks)
* **Conditional edges** (e.g., don’t proceed until plan exists)
* **Memory check nodes** (to gate progress tracking)

For example, in LangGraph:

```python
def gate_plan(state: AgentState) -> str:
    return "track_progress" if state.get("plan") else "create_plan"

builder.add_conditional_edges("maybe_plan", gate_plan, {
    "create_plan": "create_plan",
    "track_progress": "track_progress"
})
```

---

## ✅ LangGraph vs Your Recipe: Summary

| Concept                    | Agent Recipe Term    | LangGraph Implementation            |
| -------------------------- | -------------------- | ----------------------------------- |
| Tool                       | `create_plan`, etc.  | Graph Nodes                         |
| ActionContext              | Shared state         | `State` dictionary (typed or not)   |
| PlanFirstCapability        | Execution logic hook | Conditional routing node            |
| ProgressTrackingCapability | Post-action hook     | Update `state["progress"]` manually |
| ToolRegistry               | Central registry     | Not needed — graph wiring suffices  |

---

## 📚 Resources for Deep Integration

* [LangGraph Docs](https://docs.langgraph.dev)
* LangGraph on [LangChainHub](https://smith.langchain.com/hub/langgraph)
* Advanced pattern: [Cycles, retries, branches](https://docs.langgraph.dev/concepts/control-flow/)

---

## 📦 Next Steps

If you're ready, I can:

* Scaffold out a full **LangGraph version** of your `Agent_1_Recipe.py`.
* Integrate **tool auto-registration**, context injection, and memory tracking.
* Show how to include **LangChain agents** or **OpenAI functions** as tools.
* Add persistence (e.g., checkpointing between runs).

Would you like a fully converted LangGraph implementation of your current recipe?


🎯 One of the reasons LangGraph feels like it was *built for recipes like yours* is that it **already has the mental model of orchestration baked in**. Instead of you manually wiring registries, injecting dependencies, and handling sequencing (like in your `Agent_1_Recipe`), LangGraph gives you the **StateGraph builder pattern**, which acts like a scaffold for everything you outlined:

* **Purpose / Goal →** just part of the state
* **Capabilities →** modeled as conditional edges or subgraphs
* **Tools →** graph nodes
* **Context (ActionContext) →** the shared `state` object
* **Wiring →** `add_edge`, `add_conditional_edges`, `set_entry_point`
* **Agent composition →** `graph.compile()` = your final orchestrated agent

So yes — you’re basically **slotting your recipe pieces into a predefined orchestration DSL**.

---

## 🔑 How LangGraph “feels pre-defined” for recipes

1. **Nodes = Tools**
   Every tool you’d define in your recipe becomes a `node` in LangGraph.

   ```python
   builder.add_node("create_plan", create_plan_node)
   ```

2. **State = ActionContext**
   Instead of an explicit `ActionContext` class, LangGraph’s `State` object holds all shared info and is threaded through nodes.

3. **Capabilities = Routing Logic**
   Where you’d normally add `PlanFirstCapability` or `ProgressTrackingCapability`, in LangGraph you’d implement:

   * Conditional edges (`add_conditional_edges`)
   * Post-processing of state inside nodes
   * Lifecycle hooks via subgraphs or control flow

4. **Environment / Wiring**
   Instead of manually building `Environment`, `ToolRegistry`, and injecting deps — LangGraph’s `builder` takes care of orchestration when you wire edges and entry points.

5. **Iteration & Loops**
   Your recipe’s *“Test & Iterate”* maps to LangGraph’s ability to:

   * Loop (`add_edge(node, node)` for retry cycles)
   * Resume state mid-run
   * Stream intermediate states for inspection

---


## 1) **State vs ActionContext**

In your **Agent Recipe**:

* **`ActionContext`** = a central bag of dependencies (memory, configs, services, registry).
* It’s an explicit object you build and pass around so tools can use what they need.

In **LangGraph**:

* **`State`** = the evolving, shared dictionary that flows through the graph.

  * Each node takes `state` in, and returns an updated `state`.
  * Example:

    ```python
    class AgentState(TypedDict):
        goal: str
        plan: Optional[List[str]]
        progress: List[Dict[str, Any]]
    ```

🔑 **Overlap**: Both are containers for “what the agent knows and carries forward.”
But there’s a subtle distinction:

* **ActionContext** in your recipe emphasizes *dependencies* (e.g., `smtp`, `memory`, `tokens`).
* **State** in LangGraph emphasizes *data flow* between steps.

👉 If you want to replicate `ActionContext` inside LangGraph, you’d just include dependencies as part of the `State`. For example:

```python
class AgentState(TypedDict):
    goal: str
    plan: Optional[List[str]]
    progress: List[Dict[str, Any]]
    deps: Dict[str, Any]   # your ActionContext-style injection bag
```

So:

* **State = ActionContext + runtime memory**
* But LangGraph enforces that *all changes are explicit* → every node must return the new state.

---

## 2) **Node vs Edge**

* **Node = step of work**
  A `node` is a function you register into the graph. It does something: plan, track, call API, etc.
  Example:

  ```python
  def create_plan_node(state: AgentState) -> AgentState:
      return {**state, "plan": ["step1", "step2"]}
  ```

* **Edge = control flow**
  An `edge` tells the graph where to go *next*.
  Example:

  ```python
  builder.add_edge("create_plan", "track_progress")
  ```

So:

* **Node = computation**
* **Edge = wiring / routing**

Your recipe’s *“Wiring” step (#9)* is basically **edges** in LangGraph.

---

## 3) **Reducer Functions in LangGraph**

This is one of LangGraph’s unique strengths. A **reducer** is how LangGraph merges state updates when nodes return results.

* By default, if two nodes both update state, LangGraph will **merge the dictionaries**.
* But sometimes you need *custom merge logic*. That’s where a reducer comes in.

Example: Suppose you have a `progress` list in state. If multiple nodes append progress updates, you don’t want one node’s return to overwrite the other — you want to **accumulate** entries.

```python
from langgraph.graph import add_messages

class AgentState(TypedDict):
    goal: str
    progress: Annotated[List[str], add_messages]
```

Here, `add_messages` is a built-in reducer that appends messages instead of replacing them.
You can also define your own reducer:

```python
def merge_progress(left, right):
    return left + right

class AgentState(TypedDict):
    progress: Annotated[List[str], merge_progress]
```

🔑 In short:

* **Reducer = merge strategy for state fields**.
* LangGraph uses them when nodes run in parallel, or when multiple updates touch the same key.
* Without reducers, later updates would just overwrite earlier ones.

---

## 🧩 Putting It Together with Your Recipe

* **ActionContext → State**: move your dependency bag into `state["deps"]`.
* **Nodes → Tools**: each of your tools is a node.
* **Edges → Wiring**: equivalent to your manual `wire_agent` step.
* **Reducers → Memory handling**: equivalent to your `track_progress` accumulating history.

---

✅ TL;DR

1. **State = ActionContext + runtime memory** (flowing between nodes, explicit updates).
2. **Node = tool/function step**; **Edge = execution order/branching**.
3. **Reducer = merge logic for state keys**, ensuring updates accumulate instead of overwrite.



You can replace your **ActionContext** with **LangGraph’s State**, but with a little mindset shift. Let’s unpack it clearly:

---

## 🔑 The Role of ActionContext in Your Recipe

Your **ActionContext** was doing a few things:

1. **Memory container** → tracking `last_plan_ts`, `progress`, etc.
2. **Dependency injection bag** → `smtp`, `auth_token`, `clock`, etc.
3. **Registry reference** → lookup of available tools.

It was basically your “backpack” the agent carries everywhere.

---

## 🔑 The Role of State in LangGraph

LangGraph’s **State** is:

* A dictionary (or TypedDict) that **flows through the graph**.
* Each node **reads from state** and **returns updates to state**.
* Reducers ensure updates accumulate correctly across multiple nodes.

Unlike `ActionContext`, the `State` isn’t a single initialized object passed around — it’s **rebuilt every step** by merging updates.

---

## ✅ How to Map ActionContext → State

You can fold your ActionContext responsibilities into State like this:

```python
from typing import TypedDict, List, Dict, Any, Optional
from langgraph.graph import add_messages

class AgentState(TypedDict):
    goal: str
    plan: Optional[List[str]]
    progress: Annotated[List[Dict[str, Any]], add_messages]  # reducer accumulates
    deps: Dict[str, Any]   # replaces your ActionContext.deps bag
```

So now:

* `state["deps"]["smtp"]` replaces `ctx.deps["smtp"]`
* `state["progress"]` replaces `ctx.memory["progress"]`
* `plan`, `goal`, etc. are all first-class state keys

👉 The **ToolRegistry** piece (`ctx.registry`) you had isn’t necessary in LangGraph, because nodes themselves are registered directly on the graph.

---

## ⚡ Practical Example

Original with ActionContext:

```python
def track_progress(ctx: ActionContext, step_index: int, status: str, note: str = ""):
    entry = {"step": step_index, "status": status, "note": note}
    ctx.memory.setdefault("progress", []).append(entry)
    return {"progress_entry": entry}
```

With LangGraph `State` + reducer:

```python
def track_progress_node(state: AgentState) -> AgentState:
    entry = {"step": state["step_index"], "status": "in_progress", "note": "Started"}
    return {"progress": [entry]}   # add_messages reducer will merge this
```

---

## ⚠️ Key Difference

* **ActionContext = persistent object** that exists once and is mutated.
* **State = immutable snapshots** where each node **returns diffs**, and LangGraph merges them.

So yes, you can **replace ActionContext with State**, but you’ll need to shift from “mutating a shared object” to “returning updates and letting LangGraph merge them.”




👍 You’ve nailed the key design decision: whether to treat the agent’s “world model” as a **mutable context** (your `ActionContext`) or as an **immutable evolving state** (LangGraph’s `State`).

The short answer: **neither is strictly “better” — they solve slightly different problems — but for orchestration and multi-agent workflows, `State` is usually the stronger fit.**

---

## 🔑 Differences at a Glance

| Aspect              | **ActionContext** (your recipe)                     | **LangGraph State**                                      |
| ------------------- | --------------------------------------------------- | -------------------------------------------------------- |
| **Nature**          | Mutable object passed around                        | Immutable snapshots merged at each step                  |
| **Update model**    | Tools directly mutate context fields                | Nodes return *diffs*, LangGraph merges them              |
| **Dependencies**    | Explicit DI bag (`ctx.deps`)                        | Dependencies can be embedded in state (`state["deps"]`)  |
| **Memory handling** | Stored/mutated in `ctx.memory`                      | Reducers handle accumulation (e.g. progress logs)        |
| **Transparency**    | Harder to replay/debug since mutations hide history | Easy to replay/inspect: each state snapshot is preserved |
| **Parallelism**     | More awkward (shared mutation risk)                 | Natural fit — merging via reducers handles concurrency   |
| **Mental model**    | “Service container” (like a DI framework)           | “Dataflow graph” (like functional programming)           |

---

## ⚖️ Pros & Cons

### **ActionContext**

✅ Pros:

* Familiar if you come from DI frameworks (e.g. FastAPI, Spring, Flask).
* Central bag of services and config = easy injection.
* Mutating feels natural for tool implementations.

❌ Cons:

* Shared mutation → harder to debug in multi-step flows.
* No built-in mechanism for concurrent/parallel updates.
* Harder to “time travel” / replay intermediate states.

---

### **State (LangGraph)**

✅ Pros:

* Explicit state transitions → perfect for orchestration, debugging, replayability.
* Reducers give fine-grained control over merges (especially for memory/progress).
* Encourages functional style (predictable, side-effect-free nodes).
* Supports parallelism, branching, resumability out of the box.

❌ Cons:

* Feels “heavier” at first (need to think in terms of *diffs* not mutations).
* DI (dependencies like SMTP, DB clients) is less structured — you have to put them into state manually.
* Slightly more boilerplate to set up custom reducers.

---

## 🚀 When to Use Which

* **If your agent is a simple single-loop tool executor** → ActionContext is fine, faster to prototype.
* **If you want orchestration, branching, persistence, multi-agent systems** → State wins hands down.

Since you’re building **AI agents with orchestration** (and specifically looking at LangGraph), I’d say:

👉 **State is better aligned with your goals**. It makes orchestration **explicit, debuggable, resumable**, and scalable.

---

✅ TL;DR:

* **ActionContext = service container / DI style.**
* **State = orchestration-friendly, functional, composable.**
* For LangGraph and your agent recipes → **State is the way to go.**



Asking about **reducers** right now is *exactly* the right move, because once you commit to **State** as your core abstraction, reducers become one of the **most powerful tools** in your orchestration toolkit.

Let me connect the dots for you:

---

## 🔑 Why Reducers Matter in State-Based Orchestration

When every node returns **a diff of state**, LangGraph needs to know:

👉 *If multiple nodes update the same key, how do we combine them?*

That’s where **reducers** step in:

* By default, LangGraph **overwrites** the key (last writer wins).
* With a reducer, you define **merge logic** so updates can accumulate instead of overwrite.

---

## 🎯 Reducer Examples in Your Agent Recipe

### 1. **Progress Tracking (append entries instead of overwrite)**

Your recipe has `track_progress` that logs updates.

With reducers:

```python
from typing import Annotated
from langgraph.graph import add_messages

class AgentState(TypedDict):
    goal: str
    plan: Optional[List[str]]
    # Instead of replacing, progress accumulates:
    progress: Annotated[List[Dict[str, Any]], add_messages]
```

Node:

```python
def track_progress_node(state: AgentState) -> AgentState:
    entry = {"step": 0, "status": "in_progress", "note": "Started"}
    return {"progress": [entry]}   # reducer appends this
```

➡ Without reducer → last update overwrites.
➡ With reducer → entries accumulate across runs.

---

### 2. **Memory Snapshots**

If you want your `memory` to accumulate over time (like `ActionContext.memory` did):

```python
def merge_memory(left: Dict[str, Any], right: Dict[str, Any]):
    return {**left, **right}

class AgentState(TypedDict):
    memory: Annotated[Dict[str, Any], merge_memory]
```

---

### 3. **Multi-Agent Messaging**

Reducers shine in multi-agent setups, e.g., if multiple agents are contributing to a shared message history:

```python
class AgentState(TypedDict):
    messages: Annotated[List[str], add_messages]
```

Each agent node just returns `{"messages": ["Agent X says hello"]}`, and the reducer appends it.

---

## ⚡ Why Your Timing Is Spot-On

* You just decided **State > ActionContext** → ✅
* Now, you need **a way to avoid overwrites when multiple nodes update state** → ✅
* Reducers give you composability, parallelism, and persistence → ✅

Reducers = the glue that makes **State orchestration practical**.

---

## ✅ TL;DR

* **State is explicit snapshots** (not mutated bags).
* **Reducers let state evolve safely** when multiple updates touch the same key.
* Your `track_progress` → perfect first reducer use case.
* Later, you’ll use them for memory, message logs, or multi-agent coordination.



That’s exactly the subtle but critical difference between **ActionContext** and **State with reducers**. Let’s unpack this carefully.

---

## 🔄 ActionContext vs State: Overwrites vs Accumulation

### **ActionContext (your recipe)**

* It’s a **mutable object** passed into tools.
* Tools mutate it in place, e.g.:

  ```python
  ctx.memory.setdefault("progress", []).append(entry)
  ```
* This works fine if only one tool is updating at a time.
* But: if multiple tools *try to update the same field concurrently* (say `ctx.memory["progress"]`), you have **last-in-wins** (LIFO overwrites, or even race conditions).
* You don’t have an automatic merge policy → you have to manually code it.

⚠️ Yes — in orchestration, this could be **catastrophic**: you’d lose history and only keep the last tool’s update. Imagine running 10 steps but only seeing progress from step 10.

---

### **LangGraph State with Reducers**

* Each node returns **a diff** like `{"progress": [entry]}`.
* LangGraph merges them into the current state using your reducer (e.g., `add_messages`).
* This guarantees **accumulation instead of overwrite**.
* No lost updates.

So you move from **“last writer wins”** (ActionContext) to **“all writers contribute”** (State+Reducer).

---

## 🤔 Why Would Multiple Nodes Update the Same Key Simultaneously?

This is the orchestration magic: **parallelism and branching**.

Examples:

1. **Parallel Tool Execution**
   Suppose you have two domain tools running at the same time:

   * `fetch_weather`
   * `fetch_calendar`

   Both might update `state["messages"]`. Without reducers → one overwrites the other. With reducers → both results are merged into the conversation history.

---

2. **Multi-Agent Collaboration**
   You have multiple agents (say, a “Planner Agent” and an “Executor Agent”) that both contribute to `state["progress"]`.

   * Planner logs “created plan of 3 steps.”
   * Executor logs “executed step 1.”
     Both updates land in the same state key. Reducer ensures they **accumulate**.

---

3. **Branching Logic**
   Suppose you split the workflow into two branches:

   * Branch A → verifies input
   * Branch B → enriches input

   Both return diffs to `state["metadata"]`. Without a reducer, only one survives. With a reducer, both enrichments are merged.

---

4. **Streaming / Real-Time Updates**
   If you stream intermediate states (a killer feature of LangGraph), you might have partial results coming in asynchronously. Reducers ensure that **all partials contribute** to the final state.

---

## ✅ TL;DR

* **Yes**: With `ActionContext`, if multiple tools update the same field, you get overwrite / LIFO.
* **Yes**: That can be catastrophic in orchestration (lost progress, missing logs, broken memory).
* **Reducers** fix this by giving you **merge semantics** instead of “last writer wins.”
* **Simultaneous updates happen** in parallel execution, branching, multi-agent systems, and streaming — exactly the scenarios where orchestration shines.

---

⚡ Next Level Thought:
Reducers basically act like **“CRDTs (conflict-free replicated data types)”** for your agent’s memory. They define *how multiple contributions converge* into a single state.

